In [1]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool


In [20]:
from typing import Optional, Type
import sqlite3


from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

class DoctorNameInput(BaseModel):
    doctor_name: str = Field(description="doctor name")

class GetDoctorSlots(BaseTool):
    name = "get_doctor_slots"
    description = "Useful when doctor's availability needs to be checked for booking appointments"
    args_schema: Type[BaseModel] = DoctorNameInput

    
    def get_doctor_id_by_name(self,doctor_name):
        connection = sqlite3.connect('doctor_appointments.db')
        cursor = connection.cursor()
        query = "SELECT id FROM doctors WHERE name = ?"
        cursor.execute(query, (doctor_name,))
        result = cursor.fetchone()
        connection.close()
        return result[0] if result else None

    def get_available_slots(self,doctor_name):
        doctor_id = self.get_doctor_id_by_name(doctor_name)
        if doctor_id is None:
            return {"error": f"No doctor found with name {doctor_name}"}
    
        connection = sqlite3.connect('doctor_appointments.db')
        cursor = connection.cursor()
        query = "SELECT slot_time FROM available_slots WHERE doctor_id = ?"
        cursor.execute(query, (doctor_id,))
        slots = cursor.fetchall()
        connection.close()
        return [slot[0] for slot in slots]
        
    def _run(
        self, doctor_name: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        
        return self.get_available_slots(doctor_name)

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [21]:
get_slots = GetDoctorSlots()
get_slots.invoke({"doctor_name":"Dr. NS Dhar"})

['2024-07-15 10:00', '2024-07-15 11:00', '2024-07-15 14:00']

In [2]:
#sql chain

from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///doctor_appointments.db")
print(db.dialect)
print(db.get_usable_table_names())


sqlite
['appointments', 'available_slots', 'doctors']


In [4]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
sql_query_formation_chain = create_sql_query_chain(llm, db)
response = sql_query_formation_chain.invoke({"question": "How many doctors are there"})
response

'SELECT COUNT("id") AS "Number of Doctors" FROM doctors'

In [5]:
db.run(response)

'[(3,)]'

In [33]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
sql_chain = write_query | execute_query
sql_chain.invoke({"question":  "How many doctors are there"})

'[(3,)]'

In [34]:
sql_chain.invoke({"question":  "What is the slots available for dr ns dhar"})

"[(1, '2024-07-15 10:00'), (2, '2024-07-15 11:00'), (3, '2024-07-15 14:00')]"

## Build Agents 

In [36]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, you are helpful when user needs to book appointments",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

@tool
def get_doctor_slots(doctor_name):
    """Useful when you need to get available slots of a doctor"""
    query = f"What is the slots available for {doctor_name}"
    return sql_chain.invoke({"question":query})

@tool
def create_appointments(doctor_name: str, patient_name: str, appointment_time: str):
    """Useful when you need to create appointments for a doctor"""
    # Get doctor ID
    connection = sqlite3.connect('doctor_appointments.db')
    cursor = connection.cursor()
    cursor.execute("SELECT id FROM doctors WHERE name = ?", (doctor_name,))
    result = cursor.fetchone()
    doctor_id = result[0] if result else None

    if not doctor_id:
        return f"Doctor named {doctor_name} not found."

    # Insert appointment
    cursor.execute('''
    INSERT INTO appointments (doctor_id, patient_name, appointment_time)
    VALUES (?, ?, ?)
    ''', (doctor_id, patient_name, appointment_time))

    connection.commit()
    connection.close()
    


tools = [get_doctor_slots]


In [37]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm_with_tools = llm.bind_tools(tools)


In [39]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [40]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [41]:
agent_executor.invoke({"input":"Please book a slot of dr ns dhar"})



> Entering new AgentExecutor chain...

Invoking: `get_doctor_slots` with `{'doctor_name': 'dr ns dhar'}`


[(1, '2024-07-15 10:00'), (2, '2024-07-15 11:00'), (3, '2024-07-15 14:00')]Dr. NS Dhar has the following available slots for booking:
1. 2024-07-15 10:00
2. 2024-07-15 11:00
3. 2024-07-15 14:00

Please let me know which slot you would like to book.

> Finished chain.


{'input': 'Please book a slot of dr ns dhar',
 'output': 'Dr. NS Dhar has the following available slots for booking:\n1. 2024-07-15 10:00\n2. 2024-07-15 11:00\n3. 2024-07-15 14:00\n\nPlease let me know which slot you would like to book.'}

## Appointment booking

In [43]:


response = sql_query_formation_chain.invoke({"question": "Book appointment of dr ns dhar on 2024-07-15 at 10 am "})


In [45]:
db.run(response)

"[(1, 'Dr. NS Dhar', '2024-07-15 10:00')]"

In [25]:
query = """Put appointment records to table.
Patient Name: Hrisikesh Neogi
doctor name: dr ns dhar
time: 2024-07-15 10:00

"""

response = sql_query_formation_chain.invoke({"question":query})
response

'INSERT INTO appointments (doctor_id, patient_name, appointment_time)\nSELECT doctors.id, "Hrisikesh Neogi", "2024-07-15 10:00"\nFROM doctors\nWHERE doctors.name = "Dr. NS Dhar";'

In [26]:
db.run(response)

''

In [28]:
db.run("select * from appointments")

"[(1, 1, 'Hrisikesh Neogi', '2024-07-15 10:00')]"

In [30]:
# update available times in doctor database
query = """An appointment has been booked for dr ns dhar
time: 2024-07-15 10:00
Please del the available time in table

"""

update_query = sql_query_formation_chain.invoke({"question":query})
update_query

"DELETE FROM available_slots\nWHERE doctor_id = 1 AND slot_time = '2024-07-15 10:00';"

In [31]:
#before update
db.run("select * from available_slots")

"[(1, 1, '2024-07-15 10:00'), (2, 1, '2024-07-15 11:00'), (3, 1, '2024-07-15 14:00'), (4, 2, '2024-07-15 09:00'), (5, 2, '2024-07-15 13:00'), (6, 3, '2024-07-15 15:00')]"

In [32]:
#after update
db.run(update_query)

db.run("select * from available_slots")

"[(2, 1, '2024-07-15 11:00'), (3, 1, '2024-07-15 14:00'), (4, 2, '2024-07-15 09:00'), (5, 2, '2024-07-15 13:00'), (6, 3, '2024-07-15 15:00')]"

In [33]:
#check dr ns dhar availability now
query = """What is the available slots for dr ns dhar

"""

availability_query = sql_query_formation_chain.invoke({"question":query})
db.run(availability_query)

"[('2024-07-15 11:00',), ('2024-07-15 14:00',)]"